In [1]:
import torch
from torch.utils.data import DataLoader
import numpy as np
from pathlib import Path
import open3d as o3d
from tqdm import tqdm
import yaml
import argparse
import sys
import os

# BASE_DIR = os.path.dirname(os.path.abspath(__file__))
# sys.path.append(BASE_DIR)
# sys.path.append(os.path.join(BASE_DIR, 'frustum_pointnets_pytorch'))

from dataset import FrustumDataset
from frustum_pointnets_pytorch.models.frustum_pointnets_v1 import FrustumPointNetv1
from frustum_pointnets_pytorch.models.model_util import NUM_HEADING_BIN, NUM_SIZE_CLUSTER, g_mean_size_arr
from frustum_pointnets_pytorch.models.model_util import get_box3d_corners_helper as get_box3d_corners

def get_scene_list(data_root, train_val_split=0.8, is_training=False):
    """Get list of scenes for training or validation"""
    data_root = Path(data_root)
    all_scenes = sorted([d for d in data_root.iterdir() if d.is_dir()])
    
    # Split into train and validation
    num_scenes = len(all_scenes)
    num_train = int(num_scenes * train_val_split)
    
    # Use same random seed as training for consistent splits
    np.random.seed(42)
    scene_indices = np.random.permutation(num_scenes)
    train_indices = scene_indices[:num_train]
    val_indices = scene_indices[num_train:]
    
    # Return appropriate scene list
    if is_training:
        return [all_scenes[i] for i in train_indices]
    else:
        return [all_scenes[i] for i in val_indices]

def visualize_input_pointcloud(point_cloud):
    """
    Visualize the input point cloud
    Args:
        point_cloud: (N, 4) array of points with intensity
    """
    print("\nInput Point Cloud Visualization:")
    print(f"Point cloud shape: {point_cloud.shape}")
    
    # Create Open3D visualization
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name="Input Point Cloud")
    
    # Create point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud[:, :3])
    
    # Color point cloud by intensity if available
    if point_cloud.shape[1] > 3:
        colors = np.zeros((len(point_cloud), 3))
        colors[:, 0] = point_cloud[:, 3]  # Map intensity to red channel
        colors = colors / colors.max()
        pcd.colors = o3d.utility.Vector3dVector(colors)
    
    # Add point cloud to visualization
    vis.add_geometry(pcd)
    
    # Add coordinate frame
    coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
        size=1.0,  # 1 meter size
        origin=[0, 0, 0]  # at origin
    )
    vis.add_geometry(coord_frame)
    
    # Set view control and render options
    opt = vis.get_render_option()
    opt.background_color = np.array([0.1, 0.1, 0.1])
    opt.point_size = 2.0
    
    # Set camera viewpoint
    vc = vis.get_view_control()
    vc.set_zoom(0.8)
    vc.set_lookat([0, 0, 0])
    vc.set_up([0, 0, 1])  # Set Z axis as up direction
    
    # Run visualization
    vis.run()
    vis.destroy_window()

def visualize_results(point_cloud, boxes_3d, scores, config):
    """
    Visualize detection results
    Args:
        point_cloud: (N, 4) array of points
        boxes_3d: List of 3D bounding boxes [x, y, z, l, w, h, heading]
        scores: List of confidence scores
        config: Configuration dictionary
    """
    print("\nDetection Results Visualization:")
    print(f"Point cloud shape: {point_cloud.shape}")
    print(f"Number of boxes: {len(boxes_3d)}")
    print(f"Scores: {scores}")
    print(f"First box: {boxes_3d[0] if len(boxes_3d) > 0 else 'No boxes'}")
    
    # Create Open3D visualization
    vis = o3d.visualization.Visualizer()
    vis.create_window(window_name="Detection Results")
    
    # Create point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(point_cloud[:, :3])
    
    # Color point cloud by intensity if available
    if point_cloud.shape[1] > 3:
        colors = np.zeros((len(point_cloud), 3))
        colors[:, 0] = point_cloud[:, 3]  # Map intensity to red channel
        colors = colors / colors.max()
        pcd.colors = o3d.utility.Vector3dVector(colors)
    
    # Add point cloud to visualization
    vis.add_geometry(pcd)
    
    # Add coordinate frame
    coord_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
        size=1.0,  # 1 meter size
        origin=[0, 0, 0]  # at origin
    )
    vis.add_geometry(coord_frame)
    
    # Add 3D boxes
    boxes_added = 0

    for box, score in zip(boxes_3d, scores):
        if score < config.get('min_score', 0.5):  # Add default threshold if not in config
            continue
        
        # Get box corners
        center = box[:3]
        size = box[3:6]
        heading = box[6]
        corners = get_box3d_corners(center, heading, size)
        corners = corners.reshape(-1, 3)  # Reshape to (8, 3) for visualization
        
        # Define box edges
        lines = [
            [0, 1], [1, 2], [2, 3], [3, 0],  # Bottom face
            [4, 5], [5, 6], [6, 7], [7, 4],  # Top face
            [0, 4], [1, 5], [2, 6], [3, 7]   # Connecting lines
        ]
        
        # Create line set for box
        line_set = o3d.geometry.LineSet()
        line_set.points = o3d.utility.Vector3dVector(corners)
        line_set.lines = o3d.utility.Vector2iVector(lines)
        
        # Set all lines to red color
        colors = [[1, 0, 0] for _ in range(len(lines))]  # All lines red
        line_set.colors = o3d.utility.Vector3dVector(colors)
        
        vis.add_geometry(line_set)
        boxes_added += 1
    
    print(f"Total boxes added to visualization: {boxes_added}")
    
    # Set view control and render options
    opt = vis.get_render_option()
    opt.background_color = np.array(config.get('background_color', [0.1, 0.1, 0.1]))
    opt.point_size = config.get('point_size', 2.0)
    opt.line_width = config.get('line_width', 5.0)  # Thicker lines like in bbox_pred_viz
    
    # Set camera viewpoint
    vc = vis.get_view_control()
    vc.set_zoom(0.8)
    vc.set_lookat([0, 0, 0])
    vc.set_up([0, 0, 1])  # Set Z axis as up direction
    
    # Run visualization
    vis.run()
    vis.destroy_window()

def get_boxes_from_predictions(predictions):
    """
    Convert model predictions to 3D boxes and scores
    Args:
        predictions: dict containing model predictions
    Returns:
        boxes_3d: List of 3D bounding boxes [x, y, z, l, w, h, heading]
        scores: List of confidence scores
    """
    print("\nDebug predictions:")
    for k, v in predictions.items():
        if isinstance(v, torch.Tensor):
            print(f"{k}: shape {v.shape}, range [{v.min().item():.3f}, {v.max().item():.3f}]")
    
    # Get box centers
    centers = predictions['box3d_center'].detach().cpu().numpy()  # (bs, 3)
    
    # Get heading information
    heading_scores = torch.softmax(predictions['heading_scores'], dim=1).detach().cpu().numpy()  # (bs, NH)
    heading_residuals = predictions['heading_residual'].detach().cpu().numpy()  # (bs, NH)
    heading_class = np.argmax(heading_scores, axis=1)  # (bs,)
    heading_angles = heading_class * (2 * np.pi / NUM_HEADING_BIN) + \
                    np.array([heading_residuals[i, heading_class[i]] for i in range(len(heading_class))])  # (bs,)
    
    # Get size information
    size_scores = torch.softmax(predictions['size_scores'], dim=1).detach().cpu().numpy()  # (bs, NS)
    size_residuals = predictions['size_residual'].detach().cpu().numpy()  # (bs, NS, 3)
    size_class = np.argmax(size_scores, axis=1)  # (bs,)
    
    # Get predicted sizes
    mean_sizes = g_mean_size_arr[size_class]  # (bs, 3)
    size_residuals_for_class = np.array([size_residuals[i, size_class[i]] for i in range(len(size_class))])  # (bs, 3)
    sizes = mean_sizes + size_residuals_for_class  # (bs, 3)
    
    print("\nDebug box calculations:")
    print(f"Centers shape: {centers.shape}, range [{centers.min():.3f}, {centers.max():.3f}]")
    print(f"Sizes shape: {sizes.shape}, range [{sizes.min():.3f}, {sizes.max():.3f}]")
    print(f"Heading angles shape: {heading_angles.shape}, range [{heading_angles.min():.3f}, {heading_angles.max():.3f}]")
    
    # Combine into boxes_3d format [x, y, z, l, w, h, heading]
    boxes_3d = np.concatenate([centers, sizes, heading_angles[:, np.newaxis]], axis=1)  # (bs, 7)
    
    # Get confidence scores from segmentation logits
    seg_scores = torch.softmax(predictions['logits'], dim=2)  # [bs, N, 2]
    seg_conf = seg_scores[:, :, 1].mean(dim=1).detach().cpu().numpy()  # Average foreground score
    
    # Get heading confidence (max of softmaxed scores)
    heading_conf = np.max(heading_scores, axis=1)
    
    # Get size confidence (max of softmaxed scores)
    size_conf = np.max(size_scores, axis=1)
    
    # Combine scores - using geometric mean instead of minimum
    scores = np.power(seg_conf * heading_conf * size_conf, 1/3)
    
    print(f"\nConfidence Scores Breakdown:")
    print(f"Segmentation confidence: {seg_conf}")
    print(f"Heading confidence: {heading_conf}")
    print(f"Size confidence: {size_conf}")
    print(f"Final combined score: {scores}")
    
    print(f"\nFinal outputs:")
    print(f"Boxes shape: {boxes_3d.shape}")
    print(f"Scores shape: {scores.shape}, range [{scores.min():.3f}, {scores.max():.3f}]")
    
    return boxes_3d, scores

def main():

    file_config = 'configs/test_config.yaml'
    # Load config
    with open(file_config, 'r') as f:
        config = yaml.safe_load(f)
    
    # Get validation scene list
    val_scenes = get_scene_list(config['data_root'], config['train_val_split'], is_training=False)
    print(f"Found {len(val_scenes)} validation scenes")
    
    # Initialize detector
    detector = FrustumPointNetv1(
        n_classes=config.get('num_classes', 3),
        n_channel=config.get('num_channels', 3)
    ).to(config['device'])
    
    # Load weights
    if config['weights_path']:
        weights = torch.load(config['weights_path'], map_location=config['device'])
        if 'model_state_dict' in weights:
            detector.load_state_dict(weights['model_state_dict'])
        else:
            detector.load_state_dict(weights)  # Handle case where weights are saved directly
    detector.eval()
    
    # Create dataset and dataloader
    dataset = FrustumDataset(
        data_path=config['data_root'],
        scene_list=val_scenes,
        num_points=config['num_points']
    )
    
    dataloader = DataLoader(
        dataset,
        batch_size=1,  # Use batch size 1 for testing
        shuffle=False,
        num_workers=config['num_workers']
    )
    
    # Run inference
    with torch.no_grad():
        for batch in tqdm(dataloader, desc='Testing'):
            # Move data to device
            data_dict = {k: v.to(config['device']) if isinstance(v, torch.Tensor) else v 
                        for k, v in batch.items()}
            
            # Visualize input point cloud
            input_points = data_dict['point_cloud'][0].cpu().numpy().transpose(1, 0)  # (N, 4)
            visualize_input_pointcloud(input_points)
            
            # Run detection
            predictions = detector(data_dict)
            
            # Convert predictions to boxes and scores
            boxes_3d, scores = get_boxes_from_predictions(predictions)
            
            # Get point cloud for visualization
            points = data_dict['point_cloud'][0].cpu().numpy()  # (4, N) -> (N, 4)
            points = points.transpose(1, 0)
            
            # Visualize results
            visualize_results(points, boxes_3d, scores, config)
            
            # Break after first batch if in debug mode
            if config.get('debug', False):
                break

if __name__ == '__main__':
    main() 

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


NameError: name '__file__' is not defined